<a href="https://colab.research.google.com/github/duybluemind1988/Data-science/blob/master/Sonion/Compare_all_bigdata_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.datarevenue.com/en-blog/pandas-vs-dask-vs-vaex-vs-modin-vs-rapids-vs-ray

# Modin

In [ ]:
#!pip install modin[all] # Install all of the above

In [18]:
import datetime
from dateutil.parser import parse

In [3]:
import os
import modin.pandas as pd
path='/content/drive/My Drive/Data/Sonion/V04R-V04R-SQLData.dat' # 1.88 gb

In [4]:
%%time
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask
df_dask = pd.read_csv(path,sep='\t',header=None)

CPU times: user 2.16 s, sys: 1.55 s, total: 3.71 s
Wall time: 59.1 s


In [ ]:
#import pandas as pd
#df = pd.read_csv(f, parse_dates={'Date': [0,1,2]}, index_col='Date')
#df = pd.read_csv("f500_new.csv" , encoding = "iso-8859-1" , 
#parse_dates = {'date_col' : ["day", "month", "year"]} )
#df['date_col'] = pd.to_datetime(df['date_col'])

#import pandas as pd
#mydateparser = lambda x: pd.datetime.strptime(x, "%Y %d %m %H:%M:%S")
#df = pd.read_csv("f500_new.csv" , encoding = "iso-8859-1", parse_dates = ['date_col'] , 
#date_parser = mydateparser)

In [5]:
%%time
df_dask.head()

CPU times: user 29 ms, sys: 8.91 ms, total: 37.9 ms
Wall time: 2.44 s


,0,1,2,3,4,5,6,7,8,9,10,125,126,127,128,129,130,131,132,133,134,135
0,DEAFAULT,1,33AE015,11,0.0477,12/22/2017 8:44:4,1,0,300,2.34,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
1,DEAFAULT,1,33ae015,11,0.0477,12/22/2017 8:44:53,1,0,300,2.35,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
2,DEAFAULT,1,33ae015,11,0.0477,12/22/2017 8:45:15,1,0,300,2.33,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
3,DEAFAULT,1,33ae015,11,0.0477,12/22/2017 8:45:54,1,0,300,2.32,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
4,DEAFAULT,1,33ae015,11,0.0477,12/22/2017 8:46:28,1,0,300,2.36,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN


In [6]:
%%time
os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
df_ray = pd.read_csv(path,sep='\t',header=None)

CPU times: user 2.05 s, sys: 1.54 s, total: 3.59 s
Wall time: 55.4 s


In [7]:
%%time
df_ray.head()

CPU times: user 601 µs, sys: 0 ns, total: 601 µs
Wall time: 685 µs


,0,1,2,3,4,5,6,7,8,9,10,125,126,127,128,129,130,131,132,133,134,135
0,DEAFAULT,1,33AE015,11,0.0477,12/22/2017 8:44:4,1,0,300,2.34,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
1,DEAFAULT,1,33ae015,11,0.0477,12/22/2017 8:44:53,1,0,300,2.35,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
2,DEAFAULT,1,33ae015,11,0.0477,12/22/2017 8:45:15,1,0,300,2.33,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
3,DEAFAULT,1,33ae015,11,0.0477,12/22/2017 8:45:54,1,0,300,2.32,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
4,DEAFAULT,1,33ae015,11,0.0477,12/22/2017 8:46:28,1,0,300,2.36,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN


Cả dask và ray đều dùng ram để đọc, sau đó giải phóng ram về ban đầu (mất khoảng gần 2gb ram)

## Dask

In [9]:
%%time
df_dask[2].value_counts()

CPU times: user 60.3 ms, sys: 24.4 ms, total: 84.7 ms
Wall time: 1.63 s


SU3037_0112       2331513
SU3032_0112        521965
SU3036_0110        208614
CP_SU3032_0112      13088
su3037_0112          7095
su3032_0112          1726
su3036_0110           533
SU3037_0112t          342
SU2010_0112           305
5820                   75
cp_su3032_0112         50
tSU3037_0112           45
33ae015                10
su3037_0112t            3
5070                    3
EST65DB01               2
t-su_ritep01            1
CP_SU3037_0112          1
33AE015                 1
dtype: int64

In [12]:
%%time
df_product_dask=df_dask[df_dask[2]=='SU3037_0112']

CPU times: user 1.23 s, sys: 93 ms, total: 1.32 s
Wall time: 2.54 s


In [16]:
df_product_dask.head()

,0,1,2,3,4,5,6,7,8,9,10,125,126,127,128,129,130,131,132,133,134,135
11,DEFAULT,58536826 medium m1 wk3,SU3037_0112,7117,0.01349,1/12/2018 14:6:39,1,0,933,2.82,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
12,DEFAULT,58536826 medium m1 wk3,SU3037_0112,7117,0.01349,1/12/2018 14:6:53,1,0,933,3.00,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
13,DEFAULT,58536826 medium m1 wk3,SU3037_0112,7117,0.01349,1/12/2018 14:7:1,1,0,933,2.96,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
14,DEFAULT,58536826 medium m1 wk3,SU3037_0112,7117,0.01349,1/12/2018 14:7:18,1,0,933,5.35,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
15,DEFAULT,58536826 medium m1 wk3,SU3037_0112,7117,0.01349,1/12/2018 14:7:22,1,0,933,3.40,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN


In [21]:
%%time
len(df_product_dask)

CPU times: user 67 µs, sys: 21 µs, total: 88 µs
Wall time: 92.3 µs


2331513

In [27]:
%%time
df_product_dask[5].astype('M8[us]')

CPU times: user 21.7 ms, sys: 4.44 ms, total: 26.1 ms
Wall time: 70.9 ms


RayTaskError: ignored

In [28]:
%%time
meta = ('time', pd.Timestamp)
df_product_dask[5].map_partitions(pd.to_datetime, meta=meta)

AttributeError: ignored

In [ ]:
#my_ddf['time'].map_partitions(to_datetime, columns='time').compute(get=dask.async.get_sync)

In [ ]:
#%time to_datetime(dask_df['time'].compute(), format="%Y%m%d %H:%M:%S.%f" )

#%time to_datetime(pandas_df.time, format="%Y%m%d %H:%M:%S.%f")

In [ ]:
#%%time
#df_product_dask[5]=df_product_dask[5].apply(lambda x: parse(x) )
#df_product_dask[5]=df_product_dask[5].apply(lambda x: x.date() )

## Ray

In [10]:
%%time
df_ray[2].value_counts()

CPU times: user 60.2 ms, sys: 24 ms, total: 84.2 ms
Wall time: 1.47 s


SU3037_0112       2331513
SU3032_0112        521965
SU3036_0110        208614
CP_SU3032_0112      13088
su3037_0112          7095
su3032_0112          1726
su3036_0110           533
SU3037_0112t          342
SU2010_0112           305
5820                   75
cp_su3032_0112         50
tSU3037_0112           45
33ae015                10
su3037_0112t            3
5070                    3
EST65DB01               2
t-su_ritep01            1
CP_SU3037_0112          1
33AE015                 1
dtype: int64

In [14]:
%%time
df_product_ray=df_ray[df_ray[2]=='SU3037_0112']

CPU times: user 1.3 s, sys: 93.8 ms, total: 1.39 s
Wall time: 2.94 s


In [17]:
df_product_ray.head()

,0,1,2,3,4,5,6,7,8,9,10,125,126,127,128,129,130,131,132,133,134,135
11,DEFAULT,58536826 medium m1 wk3,SU3037_0112,7117,0.01349,1/12/2018 14:6:39,1,0,933,2.82,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
12,DEFAULT,58536826 medium m1 wk3,SU3037_0112,7117,0.01349,1/12/2018 14:6:53,1,0,933,3.00,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
13,DEFAULT,58536826 medium m1 wk3,SU3037_0112,7117,0.01349,1/12/2018 14:7:1,1,0,933,2.96,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
14,DEFAULT,58536826 medium m1 wk3,SU3037_0112,7117,0.01349,1/12/2018 14:7:18,1,0,933,5.35,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
15,DEFAULT,58536826 medium m1 wk3,SU3037_0112,7117,0.01349,1/12/2018 14:7:22,1,0,933,3.40,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN


In [22]:
%%time
len(df_product_ray)

CPU times: user 27 µs, sys: 8 µs, total: 35 µs
Wall time: 39.1 µs


2331513

In [20]:
%%time
df_product_ray[5]=df_product_ray[5].apply(lambda x: parse(x) )
df_product_ray[5]=df_product_ray[5].apply(lambda x: x.date() )

KeyboardInterrupt: ignored